In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
results = search_tool.invoke("Who's the current President of France?")
print(results)

The current president is Emmanuel Macron , who succeeded François Hollande on 14 May 2017 , and was inaugurated for a second term on 7 May 2022. The current president is Emmanuel Macron , who succeeded François Hollande on 14 May 2017 following the 2017 presidential election , and was ... Emmanuel Macron, France ’ s former economy minister, who led the first round of presidential elections on April 23, will very likely be the Fifth ... François Gérard Georges Hollande (born 12 August 1954) is a French politician who is the President -elect of France . Emmanual Macron, the current President of France . ... sworn to block employment of cheap labor from eastern Europe in an attempt to secure the France ...


making tool example

In [2]:
from langchain_core.tools import Tool
import random

def get_weather(location: str)-> str:
    weather_conditions=[
        {"condition": "RAINY", "temp_c":15},
        {"condition": "CLEAR", "temp_c":25},
        {"condition": "WINDY", "temp_c":20},
    ]
    data= random.choice(weather_conditions)
    return f"Weather in {location} :{data['condition']}, {data['temp_c']} C"

weather_info_tool= Tool(
    name="get_weateher",
    func= get_weather,
    description="Fetches dummy weather information for a given location."
)                    
print(weather_info_tool.invoke("Kathmandu"))

Weather in Kathmandu :WINDY, 20 C


In [3]:
from huggingface_hub import list_models

def get_hub_stats(author: str) -> str:
    try:
        models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

        if models:
            model = models[0]
            model_id = getattr(model, "modelId", getattr(model, "id", "Unknown"))
            downloads = getattr(model, "downloads", "Unknown")
            return f"The most downloaded model by {author} is {model_id} with {downloads:,} downloads"
        else:
            return f"No models found for author {author}."
    except Exception as e:
        return f"Error fetching models for {author}: {str(e)}"

hub_stats_tool = Tool(
    name="get_hub_stats",
    func=get_hub_stats,
    description="Fetches the most downloaded model from a specific author on the Hugging Face Hub."
)

print(hub_stats_tool.invoke("facebook"))

The most downloaded model by facebook is facebook/contriever with 7,617,234 downloads


Integrating the tools

In [4]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
import sys

load_dotenv()

try:
    API_KEY = os.environ["GEMINI_PAID_KEY"]
except KeyError:
    print("Error: The 'gemini_api_key' environment variable is not set.")
    sys.exit(1)

genai.configure(api_key=API_KEY)

model = genai.GenerativeModel(
    model_name="gemini-2.5-pro",        
)

In [5]:
# import os
# from langchain_community.tools import DuckDuckGoSearchRun
# from typing import TypedDict, List
# from langgraph.graph import StateGraph, START
# from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
# from dotenv import load_dotenv
# import google.generativeai as genai
# from google.genai import types

# load_dotenv()
# try:
#     GEMINI_API_KEY = os.environ["gemini_api_key"]
# except KeyError:
#     print("Error: The 'gemini_api_key' environment variable is not set.")
#     exit(1)

# genai.configure(api_key=GEMINI_API_KEY)
# model = genai.GenerativeModel(model_name="gemini-2.5-pro")

# class AgentState(TypedDict):
#     messages: List[AnyMessage]

# MAX_HISTORY = 3

# def call_gemini(prompt: str, state: AgentState) -> AgentState:
#     last_messages = state["messages"][-MAX_HISTORY:]
#     contents = []

#     for msg in last_messages:
#         role = "assistant" if isinstance(msg, AIMessage) else "user"
#         text = msg.content[:500]
#         contents.append(
#             types.Content(
#                 role=role,
#                 parts=[types.Part(text=text)]
#             )
#         )

#     contents.append(
#         types.Content(
#             role="user",
#             # Ensure we use 'prompt' here
#             parts=[types.Part(text=prompt[:1000])]
#         )
#     )

#     response = model.generate_content(
#         contents=contents,
#         generation_config=types.GenerateContentConfig(
#             max_output_tokens=500
#         )
#     )

#     ai_text = response.text.strip()
#     ai_msg = AIMessage(content=ai_text)
#     return {"messages": [ai_msg]}

# def assistant(state: AgentState) -> AgentState:
#     messages_content = []
#     for msg in state["messages"][-MAX_HISTORY:]:
#         try:
#             content = msg.content
#             if content is not None:
#                 messages_content.append(content)
#         except AttributeError:
#             continue
            
#     last_messages = "\n".join(messages_content)
    
#     prompt = f"As an assistant, respond to the following user query. The conversation history is:\n\n{last_messages}"
#     return call_gemini(prompt, state)

# tools = {
#     "search_tool": search_tool,
#     "weather_tool": weather_info_tool,
#     "hub_stats_tool": hub_stats_tool
# }

# def route_tool(state: AgentState) -> str:
#     last_msg = state["messages"][-1].content.lower()
#     if "facebook" in last_msg or "popular model" in last_msg:
#         return "hub_stats_tool"
#     elif "weather" in last_msg:
#         return "weather_tool"
#     elif "who" in last_msg or "search" in last_msg:
#         return "search_tool"
#     else:
#         return "assistant"

# builder = StateGraph(AgentState)
# builder.add_node("assistant", assistant)

# for tool_name, tool_func in tools.items():
#     builder.add_node(tool_name, tool_func)

# builder.add_edge(START, "assistant")

# builder.add_conditional_edges("assistant", route_tool, {
#     "search_tool": "search_tool",
#     "weather_tool": "weather_tool",
#     "hub_stats_tool": "hub_stats_tool",
#     "assistant": "assistant"
# })

# for tool_name in tools.keys():
#     builder.add_edge(tool_name, "assistant")

# alfred = builder.compile()

# messages = [HumanMessage(content="Who is Facebook and what's their most popular model?")]
# response = alfred.invoke({"messages": messages})

# print("Alfred's Response:")
# print(response["messages"][-1].content)

In [8]:
import os
import random
from typing import TypedDict, List
from langgraph.graph import StateGraph, START
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_google_genai import ChatGoogleGenerativeAI
from huggingface_hub import list_models
from dotenv import load_dotenv

load_dotenv()

try:
    GEMINI_API_KEY = os.environ["gemini_api_key"]
except KeyError:
    print("Error: The 'gemini_api_key' environment variable is not set.")
    exit(1)

genai.configure(api_key=GEMINI_API_KEY)

llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-pro", 
        max_output_tokens=500
)


class AgentState(TypedDict):
    """Represents the state of the agent in the graph."""
    messages: List[AnyMessage]

tools_map = {
    "search_tool": search_tool,
    "weather_tool": weather_info_tool,
    "hub_stats_tool": hub_stats_tool
}

def call_llm(state: AgentState) -> AgentState:
    """The assistant node, which calls the LLM with the message history."""
    
    messages_to_send = [SystemMessage(content="You are a helpful multi-agent assistant.")] + state["messages"]

    response_message = llm.invoke(messages_to_send)
    
    return {"messages": [response_message]}

assistant = call_llm


def tool_node_factory(tool_name: str, tool_func: Tool):
    """Generates a wrapper node function for a given tool."""
    def wrapper(state: AgentState) -> AgentState:
        query = state["messages"][-1].content
        
        # Invoke the tool
        tool_output = tool_func.invoke(query)
        
        # Create an AIMessage containing the tool's result
        ai_msg = AIMessage(content=f"Tool ({tool_name}) Result: {tool_output}")
        
        # Return ONLY the new message for LangGraph to append
        return {"messages": [ai_msg]}
    return wrapper

search_tool_node = tool_node_factory("search_tool", search_tool)
weather_tool_node = tool_node_factory("weather_tool", weather_info_tool)
hub_stats_tool_node = tool_node_factory("hub_stats_tool", hub_stats_tool)


def route_tool(state: AgentState) -> str:
    """Decides which tool to call next based on the last message."""
    last_msg = state["messages"][-1].content.lower()
    
    if "facebook" in last_msg or "popular model" in last_msg or "meta" in last_msg:
        return "hub_stats_tool"
    elif "weather" in last_msg or "forecast" in last_msg:
        return "weather_tool"
    elif "who is" in last_msg or "search" in last_msg:
        return "search_tool"
    else:
        return "assistant"


builder = StateGraph(AgentState)
builder.add_node("assistant", assistant)

tool_nodes = {
    "search_tool": search_tool_node,
    "weather_tool": weather_tool_node,
    "hub_stats_tool": hub_stats_tool_node
}
for tool_name, tool_func in tool_nodes.items():
    builder.add_node(tool_name, tool_func)

builder.add_edge(START, "assistant")

builder.add_conditional_edges("assistant", route_tool, {
    "search_tool": "search_tool",
    "weather_tool": "weather_tool",
    "hub_stats_tool": "hub_stats_tool",
    "assistant": "assistant"
})

for tool_name in tool_nodes.keys():
    builder.add_edge(tool_name, "assistant")

alfred = builder.compile()

messages = [HumanMessage(content="Who is Facebook and what's their most popular model?")]
print("Running Alfred Agent...")

response = alfred.invoke({"messages": messages})

print("\n--- Alfred's Final Response ---")
print(response["messages"][-1].content)
print("-------------------------------\n")

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.